Panel provides a wide range of components for easily composing panels, apps, and dashboards both in the notebook and as standalone apps. The components can be broken down into three broad classes of objects:

* ``Pane`` objects allow wrapping external objects such as Bokeh, Plotly, Vega or HoloViews plots to embed them in a panel.
* ``Widget`` objects provide controls which can trigger events either in Python or Javascript.
* ``Panel`` layout objects allow combining plots into a ``Row``, ``Column``, ``Tabs`` or a ``Grid``.

All objects share an API that makes it easy to [link](./Links.ipynb) and [display and export](./Deploy_and_Export.ipynb) them. To display any panel objects in a notebook environment ensure you load the extension first:

In [ ]:
import panel as pn
pn.extension()

Note that to use certain components such as Vega, LaTeX and Plotly plots in a notebook the models must be loaded using the extension, e.g.:

    pn.extension('vega', 'katex')
    
will ensure that the Vega and LaTeX JS dependencies are loaded. Once the extension is loaded panel objects will display themselves in the notebook, outside the notebook objects can be displayed in a server using the show method or run from the commandline by appending ``.serveable()`` to the objects to be displayed.

## Panes

``Pane`` objects makes it possible to display and arrange a wide range of plots and other media on a page, including plots (matplotlib, bokeh, vega/altair, holoviews, and plotly), images (pngs, svgs, gifs, jpegs), and various Markup languages including Markdown, HTML and LaTeX.

There are two main ways to construct panes, the first is to explicitly construct a pane:

In [ ]:
pn.pane.Markdown("""
# H1
## H2
### H3
""")

Panel also provides a convenient helper function which will convert objects into a Pane or Panel layout. This utility is also used internally when passing external objects to a Panel layout such as a Row or Column. The utility resolves the appropriate representation for an object using by checking all pane objects and then taking ranking them by priority. When passing a string for instance there are many representations, however the PNG pane takes precedence if the string is a valid URL or local file path.

In [ ]:
png = pn.panel('https://upload.wikimedia.org/wikipedia/commons/4/47/PNG_transparency_demonstration_1.png', width=300)

png

To see the type of the pane use the ``pprint`` method:

In [ ]:
png.pprint()

All panel objects store the object they are wrapping on the ``object`` parameter, by setting it existing views (both in the notebook and on server instances) of this object will update:

In [ ]:
png.object = 'https://upload.wikimedia.org/wikipedia/commons/3/39/PNG_demo_heatmap_Banana.png'

In addition to the ``object`` parameter each pane type may have additional parameters which can modify how the ``object`` is rendered.

## Widgets

``Widget`` components like all objects in Panel sync their parameter state between all views of the object. Widget objects share a ``value`` parameter, layout parameters and other parameters specific to each widget. In the notebook we can display it just like other panel objects:

In [ ]:
widget = pn.widgets.TextInput(name='A widget', value='A string')
widget

In this the widget values can easily be accessed and set:

In [ ]:
widget.value = '3'
widget.width = 100

As well as linked to other objects:

In [ ]:
string = pn.pane.Str()

widget.jslink(string, value='object')

pn.Row(widget, string)

For more details about linking widgets to other objects either in Python or in Javascript see the [Links user guide](./Links.ipynb).

## Panel layouts

The third type of component is are the Panel layout objects which allow arranging widget and pane objects into everything from a simple app to a complex dashboard. There are four main types of Layout containers:

* **``Row``**: A ``Row`` arranges a list of components horizontally.
* **``Column``**: A ``Column`` arranges a list of components vertically.
* **``Tabs``**: ``Tabs`` lay out components in a list of selectable tabs. 
* **``GridSpec``**: A ``GridSpec`` lays out components on a grid.

In addition to these layout containers ``Spacer`` components allow controlling the spacing between components.

### Row & Column

The ``Row``, ``Column`` and ``Tabs`` layouts behave very similarly, they are list-like, which means they have many of the same methods as a simple Python list, making it easy to add, replace and remove components using ``append``, ``extend``, ``clear``, ``insert``, ``pop``, ``remove`` and ``__setitem__``.

In [ ]:
column = pn.Column()

# Add an item
column.append('# A title')

# Add multiple items
column.extend([pn.widgets.FloatSlider(), pn.widgets.TextInput()])

# Replace the third item
column[2] = pn.widgets.Button(name='Click here')

column

### Tabs

The ``Tabs`` object shares all the same methods however, when adding or replacing items it is also possible to pass a tuple providing a custom title:

In [ ]:
from bokeh.plotting import figure

tabs = pn.Tabs()

p1 = figure(width=300, height=300)
p1.line([1, 2, 3], [1, 2, 3])

# Add a tab
tabs.append(('Slider', pn.widgets.FloatSlider()))

# Add multiple tabs
tabs.extend([
    ('Text', pn.widgets.TextInput()),
    ('Color', pn.widgets.ColorPicker())
])

# Replace a tab
tabs[0] = ('Line Plot', p1)

tabs

### GridSpec

A ``GridSpec`` is quite different from the other ``Panel`` layout types in that it isn't list-like, instead it behaves more like a 2D-array, which expands when assigned to. This makes it a very powerful means of declaring a dashboard layout with a fixed size or with responsive sizing, i.e. one that will rescale with the browser window.

To create a ``GridSpec`` we first declare it, either with a responsive ``sizing_mode`` or a fixed width and height. Once declared we can use 2D assignment to specify the index or span on indices the object in the grid should occupy. Just like an array the indexing specifies the rows first and the columns second, i.e. ``gspec[0, 1]`` would assign an object to the first row and second column.

Like the other layout types any object can be assigned to a grid, however responsive sizing modes will only work well if the object being assigned supports interactive rescaling.

**Note**: A GridSpec modifies the layout parameters of the objects that are assigned to the grid to support both fixed and responsive sizing modes.

To demonstrate the abilities let us declare a grid with a wide range of different objects, e.g. HTML panes, bokeh figures, HoloViews objects, images and widgets:

In [ ]:
import numpy as np
import holoviews as hv
import holoviews.plotting.bokeh

from bokeh.plotting import figure

fig = figure()
fig.scatter([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 1, 2, 3, 2, 1, 0, -1, -2, -3])

gspec = pn.GridSpec(sizing_mode='stretch_both', max_height=800)

gspec[0, :3] = pn.pane.HTML("A", background='#FF0000')
gspec[1:3, 0] = pn.pane.HTML("B", background='#00FF00')
gspec[1:3, 1:3] = fig
gspec[3:5, 0] = hv.Curve([1, 2, 3])
gspec[3:5, 1] = 'https://upload.wikimedia.org/wikipedia/commons/4/47/PNG_transparency_demonstration_1.png'
gspec[4:5, 2] = pn.Column(pn.widgets.FloatSlider(), pn.widgets.ColorPicker(), pn.widgets.Toggle(name='Toggle me'))

gspec

When assigning to a grid cell that is already occupied ``GridSpec`` will generate a helpful warning which highlights which objects are overlapping an where:

In [ ]:
gspec[0:3, :2] = 'Some text'

In addition to assignment we can also slice and index the ``GridSpec`` to access an individual objects or a subregion of the grid, e.g. here we will access the last row and the last two columns:

In [ ]:
gspec[-1, 1:]